In [4]:
import numpy as np
import pandas as pd
import torch
import os

In [ ]:
task = 'RE'
dataset = 'RE'
cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)

ppl_array = np.zeros(len(total_score))
for process_num in range(1,9,1): ## maximum of k process
    if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
        ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
        for index,row in ppl_df.iterrows():
            ppl_array[index] = row[0]

In [5]:
def z_score_normalize(sample_IF):
    """
    对sample_IF字典中各个'method'对应的值（键为index、值为float的字典）进行Z-score归一化。

    参数:
    sample_IF (dict): 包含多个'method'键的字典，每个'method'键对应的值为需要进行归一化处理的字典数据。

    返回:
    dict: 归一化后的字典，结构与输入的sample_IF一致，其中每个'method'键对应的值都已经完成Z-score归一化。
    """
    for method in sample_IF.keys():
        # 获取当前method对应需要归一化的值列表，保持原有顺序
        values_list = list(sample_IF[method].values())
        # 将列表转换为torch.Tensor
        tensor_value = torch.tensor(values_list).unsqueeze(1)  # 添加维度，变为二维张量

        # 计算均值和标准差
        mean_value = tensor_value.mean()
        std_value = tensor_value.std()

        # 进行Z-score归一化
        normalized_tensor = (tensor_value - mean_value) / std_value

        # 将归一化后的结果再转换回列表
        normalized_list = normalized_tensor.squeeze(1).tolist()

        # 更新原字典中当前method对应的值
        index_list = list(sample_IF[method].keys())
        normalized_dict = {}
        for index, normalized_value in zip(index_list, normalized_list):
            normalized_dict[index] = normalized_value

        sample_IF[method] = normalized_dict

    return sample_IF

In [6]:
## reverse total score
import json
import numpy as np
import pandas as pd
import torch
from src.util_func import load_yaml
task_list = [
    ['RE','RE'],
    ['ER','abt-buy'],
    ['ER','semi-text-w'],
    ['DC','hospital'],
    ['DC','beer'],
    ['DC','rayyan'],
    ['DI','amazon'],
    ['DI','walmart'],
    ['AVE','oa_mine'],
    ['CTA','SimTab'],
    ['CTA','WebTable'],
    ['ER','walmart-amazon'],
    ['ER','semi-text-c'],
    ['ER','amazon-google'],
    ['ER','wdc'],
    ['SM','CMS']
]
for task,dataset in task_list:
    yaml_path = f'script/config_{task}_{dataset}.yaml'
    config = load_yaml(yaml_path)
    cluster_num = config['cluster_num']
    cluster_per_budget = config['sample_per_cluster']
    train_file_path = config['train_file_path']
    train_file = pd.read_json(train_file_path)

    cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
    total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)
    sample_IF = torch.load(f'Influence/{task}/{dataset}/score.pkl',weights_only=False)
    sample_IF = z_score_normalize(sample_IF)

    ppl_array = np.zeros(len(total_score))
    for process_num in range(1,9,1): ## maximum of k process
        if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
            ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
            for index,row in ppl_df.iterrows():
                ppl_array[index] = row[0]

    # QuRating_Score = np.load(f'QuRating/data/select_index_main_writing/{task}-{dataset}-QuRating-score.npy',allow_pickle=True).item()
    # for key in range(len(total_score)):
    #     # total_score[i] = total_score[i] - ppl_array[i]
    #     if sample_IF['iterative'].__contains__(key):
    #         total_score[key] += sample_IF['iterative'][key]
    #     # total_score[i] = total_score[i] + 6 * QuRating_Score[i]
    for key in sample_IF['iterative'].keys():
        total_score[key] += sample_IF['iterative'][key]


    p2_choose_index = []
    count = 0
    for i in cluster_rank.keys():
        index_all = cluster_rank[i]
        sorted_score_all = total_score[index_all]
        sorted_index_all = index_all[np.argsort(-sorted_score_all)]
        # print(index_all,sorted_index_all)
        # break
        change = cluster_per_budget - len(set(index_all[:cluster_per_budget]).intersection(set(sorted_index_all[:cluster_per_budget])))
        count += change
        p2_choose_index.extend(sorted_index_all[:cluster_per_budget])
    select_IF_df = train_file.iloc[p2_choose_index]
    json.dump(select_IF_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-noIF.json'.format(task,dataset), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_82348/3228235919.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_82348/3228235919.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_82348/3228235919.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_82348/3228235919.py:43: FutureWarning: Series.__getitem__ treating keys as positions is 

In [27]:
## reverse total score
import json
from src.util_func import load_yaml
task_list = [
    # ['RE','RE'],
    # ['ER','abt-buy'],
    # ['ER','semi-text-w'],
    # ['DC','hospital'],
    # ['DC','beer'],
    # ['DC','rayyan'],
    # ['DI','amazon'],
    # ['DI','walmart'],
    # ['AVE','oa_mine'],
    # # ['CTA','SimTab'],
    # ['CTA','WebTable'],
    # ['ER','walmart-amazon'],
    ['ER','semi-text-c'],
    # ['ER','amazon-google'],
    # ['ER','wdc'],
    # # ['SM','CMS']
]
for task,dataset in task_list:
    yaml_path = f'script/config_{task}_{dataset}.yaml'
    config = load_yaml(yaml_path)
    cluster_num = config['cluster_num']
    cluster_per_budget = config['sample_per_cluster']
    train_file_path = config['train_file_path']
    train_file = pd.read_json(train_file_path)

    cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
    total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)

    ppl_array = np.zeros(len(total_score))
    for process_num in range(1,9,1): ## maximum of k process
        if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
            ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
            for index,row in ppl_df.iterrows():
                ppl_array[index] = row[0]

    QuRating_Score = np.load(f'QuRating/data/select_index_main_writing/{task}-{dataset}-QuRating-score.npy',allow_pickle=True).item()
    for i in range(len(total_score)):
        total_score[i] = total_score[i] - ppl_array[i]
        total_score[i] = total_score[i] + 6 * QuRating_Score[i]


    p2_choose_index = []
    count = 0
    for i in cluster_rank.keys():
        index_all = cluster_rank[i]
        sorted_score_all = total_score[index_all]
        sorted_index_all = index_all[np.argsort(-sorted_score_all)]
        # print(index_all,sorted_index_all)
        # break
        change = cluster_per_budget - len(set(index_all[:cluster_per_budget]).intersection(set(sorted_index_all[:cluster_per_budget])))
        count += change
        p2_choose_index.extend(sorted_index_all[:cluster_per_budget])
    select_IF_df = train_file.iloc[p2_choose_index]
    json.dump(select_IF_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-QuRating.json'.format(task,dataset), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions

In [23]:
select_IF_df

,instruction,input,output,index
3506,You are an expert in relation extraction from ...,,"{""type"": ""University""}",453
3487,You are an expert in relation extraction from ...,,"{""type"": ""University""}",1668
5337,You are an expert in relation extraction from ...,,"{""type"": ""School""}",6753
3490,You are an expert in relation extraction from ...,,"{""type"": ""University""}",3655
5338,You are an expert in relation extraction from ...,,"{""type"": ""School""}",6753
...,...,...,...,...
6051,You are an expert in relation extraction from ...,,"{""type"": ""SnookerPlayer""}",3953
6294,You are an expert in relation extraction from ...,,"{""type"": ""SnookerChamp""}",3951
6050,You are an expert in relation extraction from ...,,"{""type"": ""SnookerPlayer""}",3953
6143,You are an expert in relation extraction from ...,,"{""type"": ""TableTennisPlayer""}",4177
